# Database Functions

Each enterprise grade DBMS supports procedural programming within the database as _functions_ or _procedures_.
Each of DBMS has their own language, often with it's unique syntax.
Here you will learn the PostgreSQL variation, PL/pgSQL, which is very similar to the Oracle language, PL/SQL.


This is the general structure of the PostgreSQL function.

```SQL
CREATE FUNCTION function_name(... arguments and data types ...)
RETURNS return_datatype
as
$$
DECLARE
   ... function local variables ... ;
BEGIN
   ... body of logic ...
   
   RETURN result_value;
END;
$$
LANGUAGE plpgsql;
```   
See full documentation at: http://www.postgresql.cn/docs/10/sql-createfunction.html



For this lab, you will connect to the `dsa_student` database and create functions within your schema, i.e., your pawprint should replace SSO below.

Note in the function below the following:
 * The name is SSO.`totalReadings`
 * We have declared the function will return an integer
 * The `DECLARE ... BEGIN` is the declaration of local function variables
 * The `BEGIN ... END;` is the body of the function
 * The `$$` ... `$$` delineates the non-SQL content
 * The `LANGUAGE plpgsql` tells the PostgreSQL engine how to parse and interpret the non-SQL content.

----

```SQL
CREATE OR REPLACE FUNCTION SSO.totalReadings ()
RETURNS integer AS $$
DECLARE
    total integer;
BEGIN
   SELECT count(*) INTO total FROM SSO.survey;
   RETURN total;
END;
$$ LANGUAGE plpgsql;
```
----

 * The most important thing to notice in this function is the modification of the `SELECT` statement.
   * `INTO total`  signifies we expect a value from the select to go into the local variable.


### Using Magic Function

**Step 1: Connect to the database**

In [ ]:
# Initialize some variables
SSO="YOUR_PAWPRINT_GOES_HERE"
hostname='pgsql.dsa.lan'
database='dsa_student'

In [ ]:
# Read the Password into mem for a moment
import getpass
read_password = getpass.getpass("Type Password and hit enter")

In [ ]:
connection_string = f"postgres://{SSO}:{read_password}@{hostname}/{database}"

In [ ]:
%load_ext sql
%sql $connection_string 

**Step 2: Create a Function**

In [ ]:
%%sql 

CREATE OR REPLACE FUNCTION totalReadings()
RETURNS integer AS 
$$
DECLARE
    total integer;   -- define a variable total with a data type
BEGIN
   SELECT count(*) INTO total FROM survey;   -- Note INTO operator
   RETURN total;
END;
$$ 
LANGUAGE plpgsql;

We can now call this function as follows: 

In [ ]:
%%sql

SELECT totalReadings()

We can probe `information_schema.routines` table to retrieve the function definition.

In [ ]:
%%sql

-- select * 
select routine_schema, routine_name, routine_type, routine_definition
from information_schema.routines
where routine_name = 'totalReadings'


The above comparison `where routine_name = 'totalReadings'` is case-senstive and database the name is stored as `totalreadings`. So we will not get any data. We can execute either of the following. 

In [ ]:
%%sql

-- select * 
select routine_schema, routine_name, routine_type, routine_definition
from information_schema.routines
where routine_name = 'totalreadings'


Or

In [ ]:
%%sql

-- select * 
select routine_schema, routine_name, routine_type, routine_definition
from information_schema.routines
where routine_name ilike 'totalReadings'

## Using psql Shell (Optional)


Step 1: Connect to the database

```bash
psql -h pgsql.dsa.lan -d dsa_student
```

We can perform the same task with psql shell. 

Below is an exmaple of pasting the function into the terminal.

Note the "CREATE FUNCTION" response from the database.

```SQL
dsa_student=# CREATE OR REPLACE FUNCTION totalReadings ()
dsa_student-# RETURNS integer AS $$
dsa_student$# DECLARE
dsa_student$#     total integer;
dsa_student$# BEGIN
dsa_student$#    SELECT count(*) INTO total FROM survey;
dsa_student$#    RETURN total;
dsa_student$# END;
dsa_student$# $$ LANGUAGE plpgsql;
CREATE FUNCTION
```

Then, to execute the function we can call the function as an argument to `SELECT`


### Expected Output Example

```SQL
dsa_student=# select totalReadings();
 totalreadings
---------------
            21
(1 row)
```

## Now that you have defined a function, let's look into the catalog.

Use `\df` and `\df+` to review the functions.

Examples:

-----
```SQL
dsa_student=# \df SSO.totalreadings
                             List of functions
 Schema  |     Name      | Result data type | Argument data types |  Type
---------+---------------+------------------+---------------------+--------
 SSO | totalreadings | integer          |                     | normal
(1 row)
```


-----



```SQL
dsa_student=# \df+ SSO.totalreadings
                                                                                                    List of functions
 Schema  |     Name      | Result data type | Argument data types |  Type  | Volatility | Parallel |  Owner  | Security
| Access privileges | Language |                    Source code                     | Description
---------+---------------+------------------+---------------------+--------+------------+----------+---------+----------
+-------------------+----------+----------------------------------------------------+-------------
 SSO | totalreadings | integer          |                     | normal | volatile   | unsafe   | SSO | invoker
|                   | plpgsql  |                                                   +|
         |               |                  |                     |        |            |          |         |
|                   |          | DECLARE                                           +|
         |               |                  |                     |        |            |          |         |
|                   |          |     total integer;                                +|
         |               |                  |                     |        |            |          |         |
|                   |          | BEGIN                                             +|
         |               |                  |                     |        |            |          |         |
|                   |          |    SELECT count(*) INTO total FROM SSO.survey;+|
         |               |                  |                     |        |            |          |         |
|                   |          |    RETURN total;                                  +|
         |               |                  |                     |        |            |          |         |
|                   |          | END;                                              +|
         |               |                  |                     |        |            |          |         |
|                   |          |                                                    |
(1 row)
```

### Your Turn: Describe the function with `\df` and paste it in the cell below.

# Save your notebook, then `File > Close and Halt`